In [20]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [21]:
# Libraries
%load_ext autoreload
%autoreload 2
import index_toolkit as tk
import pandas as pd
import yfinance as yf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# Inputs
df_market_input = '^FCHI'
riskfree_input = '^TNX'
df_allocation = pd.read_csv('index_allocation.csv', index_col = 'Ticker')

In [23]:
# Fetching df_market data and risk-free rate
df_market = yf.Ticker(df_market_input).history('max')['Close']
df_market.name = df_market_input
df_market.index = df_market.index.tz_localize(None)
riskfree = yf.Ticker(riskfree_input).history('1y')['Close'].iloc[-1] / 100

# Fetching and reating a DataFrame to store historical prices for each asset
df_history = pd.DataFrame()
for ticker in df_allocation.index:
    history = yf.Ticker(ticker).history('max')['Close']
    history.name = ticker
    df_history = pd.concat([df_history, history], axis=1)
df_history.index = pd.to_datetime(df_history.index)
df_history = df_history.resample('B').last().ffill()
df_history.index = df_history.index.tz_localize(None)

In [24]:
tk.index_table(
    index=df_allocation,
    history=df_history,
).show()

               Price   Share  Valorisation Allocation Account   Type       Industry     
Ticker                                                                                  
ACA.PA          14.3   815.0    11,621.9     27.47%      PEE   STOCKS         Financials
WPEA.PA          5.2 1,750.0     9,097.9     21.51%      PEA      ETF             Global
UST.PA          70.4    60.0     4,222.3      9.98%      CTO      ETF         Technology
AEME.PA         64.1    35.0     2,242.4      5.30%      CTO      ETF             Global
MC.PA          603.6     4.0     2,414.4      5.71%      PEA   STOCKS  Consumer Cyclical
SAN.PA         103.3    15.0     1,549.8      3.66%      PEA   STOCKS        Health Care
STLAP.PA        13.6   100.0     1,357.4      3.21%      PEA   STOCKS  Consumer Cyclical
AIR.PA         129.4    10.0     1,294.2      3.06%      PEA   STOCKS        Industrials
TTE.PA          61.2    20.0     1,224.4      2.89%      PEA   STOCKS             Energy
DG.PA          109.6 

In [33]:
# Statistiques du portefeuille par positions
tk.stats_table(
    history=df_history, 
    market=df_market, 
    riskfree=riskfree, 
    index=df_allocation, 
).show()

interactive(children=(Select(description='freq', index=1, options=('D', 'B', 'W', 'M'), value='B'), Select(des…

In [31]:
tk.chart_comparison(
    history=df_history, 
    index=df_allocation
).show()

interactive(children=(Select(description='freq', index=1, options=('D', 'B', 'W', 'M'), value='B'), Select(des…